In [1]:
import os
os.chdir(r"C:\\Umass spring 20\\645\\github\\MLbasedDBMStuning\\Code\\workload_characterization\\")
from os import listdir
from os.path import isfile, join
from gpr_model import GPRNP

In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
target_path=r"C:\\Umass spring 20\\645\\github\\MLbasedDBMStuning\\Data\\"
##Reading Target files
target_df=pd.read_csv(target_path+'Target.csv')
workload=target_df.columns[0]
X_columnlabels=target_df.columns[1:13]
Y_columnlabels=target_df.columns[13:]
X_target = np.array(target_df[X_columnlabels])
y_target = np.array(target_df[Y_columnlabels])

# # X_scaler = StandardScaler(copy=False)
# # y_scaler = StandardScaler(copy=False)
# X_target = X_scaler.fit_transform(X_target)
# y_target = y_scaler.fit_transform(y_target)

In [3]:
def loadWorkloadFileNames(folder):
    onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f))]
    return onlyfiles


In [4]:
error=[]
workload_data = {}
scores = {}
pruned_file =r"C:\\Umass spring 20\\645\\github\\MLbasedDBMStuning\\Data\\WorkloadFiles\\Pruned_unscaled\\"
        #find all the files in workload
files = loadWorkloadFileNames(pruned_file)


    # Compute workload mapping data for each unique workload
for filename in files:
    if filename.endswith(".pkl"):
            workload_mtrx = pd.read_pickle(pruned_file+filename)
          #  print(filename,workload_mtrx.shape)
            cols_list = list(workload_mtrx.columns)
            X_matrix = workload_mtrx[cols_list[1:13]].to_numpy()
            y_matrix = workload_mtrx[cols_list[13:]].to_numpy()
            unique_workload=np.unique(np.array(workload_mtrx[workload]))[0]
           # print(unique_workload)
#             X_scaler.fit(X_matrix)
#             y_scaler.fit_transform(y_matrix)
            #print(X_scaler)
            workload_data[unique_workload] = {
            'X_matrix': X_matrix,
            'y_matrix': y_matrix,
            }


   
            for workload_id, workload_entry in list(workload_data.items()):
                predictions = np.empty_like(y_target)
                X_workload = workload_entry['X_matrix']
                
               # X_scaled = X_scaler.transform(X_workload)
                y_workload = workload_entry['y_matrix']
              #  y_scaled = y_scaler.transform(y_workload)
                
                for j, y_col in enumerate(y_workload.T):
            # Using this workload's data, train a Gaussian process model
            # and then predict the performance of each metric for each of
            # the knob configurations attempted so far by the target.
                    try:
                      #  print(y_col.shape,X_workload.shape)
                        y_col = y_col.reshape(-1, 1)
                    
                        model = GPRNP()
                    #print(X_scaled)
                        model.fit(X_workload, y_col, ridge=1.0)
                        gpr_result = model.predict(X_target)
                        predictions[:, j] = gpr_result.ypreds.ravel()
                    except:
                        error.append(workload_id)
                        
        
        # compute the score (i.e., distance) between the target workload
        # and each of the known workloads
            dists = np.sqrt(np.sum(np.square(np.subtract(predictions, y_target)), axis=1))
            scores[workload_id] = np.mean(dists)
            print("score is:",scores)

# Find the best (minimum) score
best_score = np.inf
best_workload_id = None
scores_info = {}
for workload_id, similarity_score in list(scores.items()):
        
    if similarity_score < best_score:
            best_score = similarity_score
            best_workload_id = workload_id
print(best_score,best_workload_id)

score is: {100.0: 180454518.98043296}
score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882}
score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885}
score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087}
score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409}
score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947}
score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432}
score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 1804545

score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0: 180454518.9804086, 127.0: 180454518.9804087, 128.0: 180454518.98043248, 13.0: 180454518.98040918, 130.0: 180454518.98040873, 133.0: 180454518.9804087, 134.0: 180454518.98043332, 135.0: 180454518.98040947, 136.0: 180454518.98043275, 137.0: 180454518.9804088, 14.0: 180454518.9804087, 140.0: 180454518.98043305, 141.0: 180454518.9804089, 142.0: 180454518.98040858, 144.0: 180454518.98040876}
score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0

score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0: 180454518.9804086, 127.0: 180454518.9804087, 128.0: 180454518.98043248, 13.0: 180454518.98040918, 130.0: 180454518.98040873, 133.0: 180454518.9804087, 134.0: 180454518.98043332, 135.0: 180454518.98040947, 136.0: 180454518.98043275, 137.0: 180454518.9804088, 14.0: 180454518.9804087, 140.0: 180454518.98043305, 141.0: 180454518.9804089, 142.0: 180454518.98040858, 144.0: 180454518.98040876, 145.0: 180454518.98043293, 147.0: 180454518.9804097, 15.0: 180454518.9804297, 150.0: 180454518.9804091, 151.0: 180454518.98040858, 152.0: 180454518.9804095, 153.0: 180454518.98043215, 154.0: 180454518.98043385, 160.0: 180454518.98040876, 162.0: 180454518.98040906, 163.0: 180454518.980434}
score is: {100

score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0: 180454518.9804086, 127.0: 180454518.9804087, 128.0: 180454518.98043248, 13.0: 180454518.98040918, 130.0: 180454518.98040873, 133.0: 180454518.9804087, 134.0: 180454518.98043332, 135.0: 180454518.98040947, 136.0: 180454518.98043275, 137.0: 180454518.9804088, 14.0: 180454518.9804087, 140.0: 180454518.98043305, 141.0: 180454518.9804089, 142.0: 180454518.98040858, 144.0: 180454518.98040876, 145.0: 180454518.98043293, 147.0: 180454518.9804097, 15.0: 180454518.9804297, 150.0: 180454518.9804091, 151.0: 180454518.98040858, 152.0: 180454518.9804095, 153.0: 180454518.98043215, 154.0: 180454518.98043385, 160.0: 180454518.98040876, 162.0: 180454518.98040906, 163.0: 180454518.980434, 164.0: 1804545

score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0: 180454518.9804086, 127.0: 180454518.9804087, 128.0: 180454518.98043248, 13.0: 180454518.98040918, 130.0: 180454518.98040873, 133.0: 180454518.9804087, 134.0: 180454518.98043332, 135.0: 180454518.98040947, 136.0: 180454518.98043275, 137.0: 180454518.9804088, 14.0: 180454518.9804087, 140.0: 180454518.98043305, 141.0: 180454518.9804089, 142.0: 180454518.98040858, 144.0: 180454518.98040876, 145.0: 180454518.98043293, 147.0: 180454518.9804097, 15.0: 180454518.9804297, 150.0: 180454518.9804091, 151.0: 180454518.98040858, 152.0: 180454518.9804095, 153.0: 180454518.98043215, 154.0: 180454518.98043385, 160.0: 180454518.98040876, 162.0: 180454518.98040906, 163.0: 180454518.980434, 164.0: 1804545

...

score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0: 180454518.9804086, 127.0: 180454518.9804087, 128.0: 180454518.98043248, 13.0: 180454518.98040918, 130.0: 180454518.98040873, 133.0: 180454518.9804087, 134.0: 180454518.98043332, 135.0: 180454518.98040947, 136.0: 180454518.98043275, 137.0: 180454518.9804088, 14.0: 180454518.9804087, 140.0: 180454518.98043305, 141.0: 180454518.9804089, 142.0: 180454518.98040858, 144.0: 180454518.98040876, 145.0: 180454518.98043293, 147.0: 180454518.9804097, 15.0: 180454518.9804297, 150.0: 180454518.9804091, 151.0: 180454518.98040858, 152.0: 180454518.9804095, 153.0: 180454518.98043215, 154.0: 180454518.98043385, 160.0: 180454518.98040876, 162.0: 180454518.98040906, 163.0: 180454518.980434, 164.0: 1804545

score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0: 180454518.9804086, 127.0: 180454518.9804087, 128.0: 180454518.98043248, 13.0: 180454518.98040918, 130.0: 180454518.98040873, 133.0: 180454518.9804087, 134.0: 180454518.98043332, 135.0: 180454518.98040947, 136.0: 180454518.98043275, 137.0: 180454518.9804088, 14.0: 180454518.9804087, 140.0: 180454518.98043305, 141.0: 180454518.9804089, 142.0: 180454518.98040858, 144.0: 180454518.98040876, 145.0: 180454518.98043293, 147.0: 180454518.9804097, 15.0: 180454518.9804297, 150.0: 180454518.9804091, 151.0: 180454518.98040858, 152.0: 180454518.9804095, 153.0: 180454518.98043215, 154.0: 180454518.98043385, 160.0: 180454518.98040876, 162.0: 180454518.98040906, 163.0: 180454518.980434, 164.0: 1804545

score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0: 180454518.9804086, 127.0: 180454518.9804087, 128.0: 180454518.98043248, 13.0: 180454518.98040918, 130.0: 180454518.98040873, 133.0: 180454518.9804087, 134.0: 180454518.98043332, 135.0: 180454518.98040947, 136.0: 180454518.98043275, 137.0: 180454518.9804088, 14.0: 180454518.9804087, 140.0: 180454518.98043305, 141.0: 180454518.9804089, 142.0: 180454518.98040858, 144.0: 180454518.98040876, 145.0: 180454518.98043293, 147.0: 180454518.9804097, 15.0: 180454518.9804297, 150.0: 180454518.9804091, 151.0: 180454518.98040858, 152.0: 180454518.9804095, 153.0: 180454518.98043215, 154.0: 180454518.98043385, 160.0: 180454518.98040876, 162.0: 180454518.98040906, 163.0: 180454518.980434, 164.0: 1804545

score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0: 180454518.9804086, 127.0: 180454518.9804087, 128.0: 180454518.98043248, 13.0: 180454518.98040918, 130.0: 180454518.98040873, 133.0: 180454518.9804087, 134.0: 180454518.98043332, 135.0: 180454518.98040947, 136.0: 180454518.98043275, 137.0: 180454518.9804088, 14.0: 180454518.9804087, 140.0: 180454518.98043305, 141.0: 180454518.9804089, 142.0: 180454518.98040858, 144.0: 180454518.98040876, 145.0: 180454518.98043293, 147.0: 180454518.9804097, 15.0: 180454518.9804297, 150.0: 180454518.9804091, 151.0: 180454518.98040858, 152.0: 180454518.9804095, 153.0: 180454518.98043215, 154.0: 180454518.98043385, 160.0: 180454518.98040876, 162.0: 180454518.98040906, 163.0: 180454518.980434, 164.0: 1804545

score is: {100.0: 180454518.98043296, 101.0: 180454518.98040882, 102.0: 180454518.98040885, 11.0: 180454518.9804087, 110.0: 180454518.980409, 111.0: 180454518.98040947, 112.0: 180454518.980432, 12.0: 180454518.9804092, 120.0: 180454518.98040915, 123.0: 180454518.98043263, 125.0: 180454518.9804085, 126.0: 180454518.9804086, 127.0: 180454518.9804087, 128.0: 180454518.98043248, 13.0: 180454518.98040918, 130.0: 180454518.98040873, 133.0: 180454518.9804087, 134.0: 180454518.98043332, 135.0: 180454518.98040947, 136.0: 180454518.98043275, 137.0: 180454518.9804088, 14.0: 180454518.9804087, 140.0: 180454518.98043305, 141.0: 180454518.9804089, 142.0: 180454518.98040858, 144.0: 180454518.98040876, 145.0: 180454518.98043293, 147.0: 180454518.9804097, 15.0: 180454518.9804297, 150.0: 180454518.9804091, 151.0: 180454518.98040858, 152.0: 180454518.9804095, 153.0: 180454518.98043215, 154.0: 180454518.98043385, 160.0: 180454518.98040876, 162.0: 180454518.98040906, 163.0: 180454518.980434, 164.0: 1804545